### Imports

In [1]:
import time
import numpy as np
import pandas as pd
from typing import Callable, Union, List, Dict, Any, Tuple
from tqdm.notebook import tqdm
from itertools import product

In [2]:
from index_utils import IndexUtil
from experiment_utils import ExperimentUtil

### Prepare index/mappings/settings

In [3]:
INDEX_NAME = 'query_analysis'

In [4]:
INDEX = IndexUtil(INDEX_NAME)

In [5]:
def get_settings():
    return {
        "number_of_shards": 1,
        "index": {
          "analysis": {
            "analyzer": {
            "stop-english_standard_analyzer": {
                "type": "custom",
                "tokenizer": "standard_tokenizer",
                "filter": [
                    "lowercase",
                    "stop-english_filter",
                ]
             },
            },
            "filter": {
              "stop-english_filter": {
                    "type": "stop",
                    "stopwords": "_english_"
              },
              "stemmer-porter2_filter": {
                    "type": "stemmer",
                    "language": "porter2"
              },
            },
            "tokenizer": {
                "standard_tokenizer": {
                    "type": "standard"
                }
            }
          }
        }
    }

In [6]:
def get_mappings(analyzer="stop-english_standard_analyzer"):
    return {
        "properties": {
            "article_id": {
                "type": "keyword"
            },
            "text": {
                "type": "text",
                "analyzer": analyzer,
            },
            "title": {
                "type": "text",
                "analyzer": analyzer,
            },
            "text_lemma": {
                "type": "text",
                "analyzer": analyzer,
            },
            "keywords": {
                "type": "text",
                "analyzer": analyzer,
            },
            "ne": {
                "type": "text",
                "analyzer": analyzer,
            },
        }
    }

In [7]:
def document_mapping_func(doc: Dict[str, Any])->Dict[str, Any]:
    return {
        'article_id': doc['uuid'],
        'text': doc['text'],
        'title': doc['title'],
        'text_lemma': doc['text_lemma'],
        'keywords': doc['keywords'],
        'ne': doc['ne'],
    }

## Queries

In [8]:
def match_query(query:str, q_type: str, fields, limit = 10):
        q = {
                "multi_match" : {
                  "query":    query, 
                  "fields": fields,
                  "type": q_type
                }
            }
        # print(q)
        res = INDEX.elastic_connector.search(
            index=INDEX.index_name,
            size=limit,
            query = q
        )
        results =  [(hit["_source"]) for hit in res['hits']['hits']]
        return results
    
def match_query_wrapper(q_type, fields):
    return lambda query, limit: match_query(query, q_type, fields, limit)

In [9]:
match_query_wrapper("best_fields", ["text^2", "title^2"])("What is life?", 2)[0]

/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'article_id': 'db2a5e6f-0a76-464e-956b-323b77148f81',
 'text': "So if you're watching me outside Stanford, welcome to you as well. Now, of course you're not gonna be able to participate in the forums with the rest of the Stanford students because you're watching this a little bit later. But there are community-based resources for asking questions out there in the world like Stack Overflow. So definitely go and check those out as well. This SwiftUI thing is brand new, literally just a few months old. So you are on the cutting edge of technology when it comes to developing applications for iOS. And as you learn this, some things are gonna be familiar to you. There's a little bit of object-oriented programming in there. This language that you're gonna learn is kind of C-like. It shares some syntax with languages like Java as well. So that'll be familiar. But you're probably gonna be learning some pretty much brand new stuff for you as well. For example, you're gonna be learning a new pro

In [10]:
def generate_grid_data(fields, xmin,xmax,Delta):
    weighted_fields = []
    for field in fields:
        weights = np.arange(xmin,xmax+Delta,Delta)
        weighted_fields.append([f"{field}^{weight}" for weight in weights])

    grid = list(product(*weighted_fields))
    return grid

### Load datasets

In [11]:
DOCUMENTS_SQUAD, QUESTIONS_SQUAD = ExperimentUtil.load_dataset('squad_10k_additional')
DOCUMENTS_SWIFT, QUESTIONS_SWIFT = ExperimentUtil.load_dataset('swift_additional')

In [12]:
DOCUMENTS_SQUAD[0]

{'title': 'Sony Music Entertainment',
 'uuid': '6e344717-f07c-4998-8968-60f3de7d868a',
 'text': 'On October 11, 2011, Doug Morris announced that Mel Lewinter had been named Executive Vice President of Label Strategy. Lewinter previously served as chairman and CEO of Universal Motown Republic Group. In January 2012, Dennis Kooker was named President of Global Digital Business and US Sales.',
 'metadata': {},
 'keywords': 'us sales morris vice universal business doug morris global digital business label vice president dennis kooker president label strategy chairman executive executive vice president lewinter ceo motown mel lewinter universal motown republic group',
 'text_lemma': 'on October 11 , 2011 , Doug Morris announce that Mel Lewinter have be name Executive Vice President of Label Strategy . Lewinter previously serve as chairman and CEO of Universal Motown Republic Group . in January 2012 , Dennis Kooker be name President of Global Digital Business and US Sales .',
 'ne': 'Doug Mo

In [13]:
DOCUMENTS_SWIFT[0]

{'text': "So, paste. Now, this almost works as is but there's one thing I always have to do when I do this is to take this curly brace right here, cut it, replace it with the word in and then paste the curly brace over here at the start. And essentially, the curly braces have to surround the entire inline function. So, that's why we move the curly brace out in front of its arguments and return type there and use this in to separate it. Okay, now, we don't need this func up here and you can see again, no errors, this was perfectly legal way to inline this function. By the way, you probably recognize this in, we used to somewhere else over here in our View with for each, it used in and had a little argument here. So, this is gonna start making a lot more sense to you once we finish up with this. And what do I mean by finish off with this? Isn't this just finished? Well, not quite because just like when we were over here, Swift was able to infer that this was type CardContent so we didn't

### Experiment

In [14]:
def test_different_query_types(documents, questions, index = INDEX_NAME):
    hits_top_10_ls = []
    hits_top_5_ls = []
    hits_top_3_ls = []
    hits_top_1_ls = []
    query_type_ls = []
    fields_ls = []

    query_types = ["best_fields", "cross_fields", "most_fields"]
    fields = ["title", "text", "text_lemma", "keywords", "ne"]
    weighted_fields = generate_grid_data(fields, 1, 2, .25)

    INDEX.delete_index()
    INDEX.create_index(mappings=get_mappings())
    INDEX.index_all_docs(documents, document_mapping_func)
    time.sleep(5)

    for q_type in tqdm(query_types):
        for w_fields in tqdm(weighted_fields):

            all_hits = ExperimentUtil.validate(index, questions, match_query_wrapper(q_type, w_fields))
            hits_10, hits_5, hit_3, hits_1 = all_hits['hits@10'], all_hits['hits@5'], all_hits['hits@3'], all_hits['hits@1']

            hits_top_10_ls.append(hits_10)
            hits_top_5_ls.append(hits_5)
            hits_top_3_ls.append(hit_3)
            hits_top_1_ls.append(hits_1)
            query_type_ls.append(q_type)
            fields_ls.append(w_fields)
            

    return pd.DataFrame.from_dict({
        'query type':query_type_ls,
        'fields':fields_ls,
        'hits@10':hits_top_10_ls,
        'hits@5':hits_top_5_ls,
        'hits@3':hits_top_3_ls,
        'hits@1':hits_top_1_ls
    })

In [ ]:
queries_types_squad = test_different_query_types(DOCUMENTS_SQUAD, QUESTIONS_SQUAD)

/opt/conda/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
queries_types_squad.sort_values("hits@10", ascending=False)

In [ ]:
queries_types_squad.to_csv("results/queries_results_squad.csv")

In [15]:
queries_types_swift = test_different_query_types(DOCUMENTS_SWIFT, QUESTIONS_SWIFT)

  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

...

  0%|          | 0/184 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

In [16]:
queries_types_swift.sort_values("hits@10", ascending=False)

,query type,fields,hits@10,hits@5,hits@3,hits@1
800,best_fields,"(title^1.25, text^1.25, text_lemma^1.5, keywor...",0.945652,0.842391,0.782609,0.570652
3158,cross_fields,"(title^1.0, text^1.0, text_lemma^1.25, keyword...",0.945652,0.853261,0.782609,0.554348
5028,cross_fields,"(title^1.75, text^1.0, text_lemma^1.25, keywor...",0.945652,0.831522,0.755435,0.548913
4403,cross_fields,"(title^1.5, text^1.0, text_lemma^1.25, keyword...",0.945652,0.831522,0.760870,0.543478
2050,best_fields,"(title^1.75, text^1.25, text_lemma^1.5, keywor...",0.945652,0.842391,0.782609,0.570652
...,...,...,...,...,...,...
647,best_fields,"(title^1.25, text^1.0, text_lemma^1.0, keyword...",0.880435,0.798913,0.733696,0.489130
2520,best_fields,"(title^2.0, text^1.0, text_lemma^1.0, keywords...",0.880435,0.809783,0.739130,0.494565
1272,best_fields,"(title^1.5, text^1.0, text_lemma^1.0, keywords...",0.880435,0.798913,0.733696,0.489130
2522,best_fields,"(title^2.0, text^1.0, text_lemma^1.0, keywords...",0.880435,0.798913,0.733696,0.489130


In [17]:
# queries_types_swift.to_csv("results/queries_results_swift.csv")

### Explore results

In [3]:
queries_swift_df = pd.read_csv('results/queries_results_swift.csv', index_col=[0])
queries_squad_df = pd.read_csv('results/queries_results_squad.csv', index_col=[0])

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/tmp/ipykernel_76835/3462665596.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


#### SWIFT

In [29]:
queries_swift_df.sort_values(['hits@10', 'hits@5', 'hits@3', 'hits@1'], ascending=False).head(10).style.set_caption("SWIFT_UI - boosted queries sorted by hits@10")

,query type,fields,hits@10,hits@5,hits@3,hits@1
22,best_fields,"('title^1.0', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.5')",0.880435,0.798913,0.733696,0.489130
647,best_fields,"('title^1.25', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.5')",0.880435,0.798913,0.733696,0.489130
1272,best_fields,"('title^1.5', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.5')",0.880435,0.798913,0.733696,0.489130
1897,best_fields,"('title^1.75', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.5')",0.880435,0.798913,0.733696,0.489130
2522,best_fields,"('title^2.0', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.5')",0.880435,0.798913,0.733696,0.489130
20,best_fields,"('title^1.0', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.0')",0.880435,0.809783,0.739130,0.494565
21,best_fields,"('title^1.0', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.25')",0.880435,0.809783,0.739130,0.494565
645,best_fields,"('title^1.25', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.0')",0.880435,0.809783,0.739130,0.494565
646,best_fields,"('title^1.25', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.25')",0.880435,0.809783,0.739130,0.494565
1270,best_fields,"('title^1.5', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.0')",0.880435,0.809783,0.739130,0.494565


In [21]:
queries_swift_df.sort_values(['hits@5', 'hits@10', 'hits@3', 'hits@1'], ascending=False).head(10).style.set_caption("SWIFT_UI - boosted queries sorted by hits@5")

,query type,fields,hits@10,hits@5,hits@3,hits@1
22,best_fields,"('title^1.0', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.5')",0.880435,0.798913,0.733696,0.489130
647,best_fields,"('title^1.25', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.5')",0.880435,0.798913,0.733696,0.489130
1272,best_fields,"('title^1.5', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.5')",0.880435,0.798913,0.733696,0.489130
1897,best_fields,"('title^1.75', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.5')",0.880435,0.798913,0.733696,0.489130
2522,best_fields,"('title^2.0', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.5')",0.880435,0.798913,0.733696,0.489130
20,best_fields,"('title^1.0', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.0')",0.880435,0.809783,0.739130,0.494565
21,best_fields,"('title^1.0', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.25')",0.880435,0.809783,0.739130,0.494565
645,best_fields,"('title^1.25', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.0')",0.880435,0.809783,0.739130,0.494565
646,best_fields,"('title^1.25', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.25')",0.880435,0.809783,0.739130,0.494565
1270,best_fields,"('title^1.5', 'text^1.0', 'text_lemma^1.0', 'keywords^2.0', 'ne^1.0')",0.880435,0.809783,0.739130,0.494565


#### SQUAD

In [36]:
queries_squad_df.sort_values(['hits@10', 'hits@5', 'hits@3', 'hits@1'], ascending=False).head(10).style.set_caption("SQUAD - boosted queries sorted by hits@10")

,query type,fields,hits@10,hits@5,hits@3,hits@1
6015,cross_fields,"('title^2.0', 'text^1.75', 'text_lemma^1.0', 'keywords^1.75', 'ne^1.0')",0.922000,0.885000,0.853000,0.748000
6017,cross_fields,"('title^2.0', 'text^1.75', 'text_lemma^1.0', 'keywords^1.75', 'ne^1.5')",0.922000,0.885000,0.853000,0.748000
6016,cross_fields,"('title^2.0', 'text^1.75', 'text_lemma^1.0', 'keywords^1.75', 'ne^1.25')",0.922000,0.885000,0.853000,0.747000
5390,cross_fields,"('title^1.75', 'text^1.75', 'text_lemma^1.0', 'keywords^1.75', 'ne^1.0')",0.922000,0.884000,0.853000,0.749000
5391,cross_fields,"('title^1.75', 'text^1.75', 'text_lemma^1.0', 'keywords^1.75', 'ne^1.25')",0.922000,0.884000,0.853000,0.748000
4765,cross_fields,"('title^1.5', 'text^1.75', 'text_lemma^1.0', 'keywords^1.75', 'ne^1.0')",0.922000,0.883000,0.852000,0.748000
4766,cross_fields,"('title^1.5', 'text^1.75', 'text_lemma^1.0', 'keywords^1.75', 'ne^1.25')",0.922000,0.883000,0.852000,0.747000
5636,cross_fields,"('title^2.0', 'text^1.0', 'text_lemma^1.0', 'keywords^1.5', 'ne^1.25')",0.922000,0.878000,0.844000,0.738000
5635,cross_fields,"('title^2.0', 'text^1.0', 'text_lemma^1.0', 'keywords^1.5', 'ne^1.0')",0.922000,0.877000,0.843000,0.736000
5540,cross_fields,"('title^1.75', 'text^2.0', 'text_lemma^1.25', 'keywords^1.75', 'ne^1.0')",0.921000,0.887000,0.853000,0.752000


In [34]:
queries_squad_df.sort_values(['hits@5', 'hits@10', 'hits@3', 'hits@1'], ascending=False).head(10).style.set_caption("SQUAD - boosted queries sorted by hits@5")

,query type,fields,hits@10,hits@5,hits@3,hits@1
5953,cross_fields,"('title^2.0', 'text^1.5', 'text_lemma^1.75', 'keywords^1.0', 'ne^1.75')",0.918000,0.891000,0.854000,0.751000
5958,cross_fields,"('title^2.0', 'text^1.5', 'text_lemma^1.75', 'keywords^1.25', 'ne^1.75')",0.918000,0.891000,0.854000,0.751000
5177,cross_fields,"('title^1.75', 'text^1.25', 'text_lemma^1.5', 'keywords^1.0', 'ne^1.5')",0.916000,0.891000,0.854000,0.751000
5888,cross_fields,"('title^2.0', 'text^1.5', 'text_lemma^1.0', 'keywords^1.5', 'ne^1.75')",0.915000,0.891000,0.855000,0.741000
5979,cross_fields,"('title^2.0', 'text^1.5', 'text_lemma^2.0', 'keywords^1.0', 'ne^2.0')",0.914000,0.891000,0.855000,0.747000
5984,cross_fields,"('title^2.0', 'text^1.5', 'text_lemma^2.0', 'keywords^1.25', 'ne^2.0')",0.914000,0.891000,0.855000,0.747000
5828,cross_fields,"('title^2.0', 'text^1.25', 'text_lemma^1.75', 'keywords^1.0', 'ne^1.75')",0.913000,0.891000,0.855000,0.743000
5803,cross_fields,"('title^2.0', 'text^1.25', 'text_lemma^1.5', 'keywords^1.0', 'ne^1.75')",0.910000,0.891000,0.845000,0.745000
5955,cross_fields,"('title^2.0', 'text^1.5', 'text_lemma^1.75', 'keywords^1.25', 'ne^1.0')",0.918000,0.890000,0.856000,0.752000
5956,cross_fields,"('title^2.0', 'text^1.5', 'text_lemma^1.75', 'keywords^1.25', 'ne^1.25')",0.918000,0.890000,0.856000,0.751000
